In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('VA_post_dig_test.csv')

In [3]:
def extract_street_addresses(dataframe, address_column):
    street_addresses = []  # Initialize an empty list to store extracted street addresses

    #Traverse the rows of the dataframe
    for index, row in dataframe.iterrows():
        full_address = row['FULL_ADDRESS'] 

        #Extract the desired part of the address using string manipulation
        #Split the full address by comma and take the first part as the street address
        street_address = full_address.split(',')[0].strip()

        #Add the street address to the list
        street_addresses.append(street_address)

    #Create a new dataframe with the extracted street addresses
    extracted_addresses_df = pd.DataFrame({
        'Extracted Street Addresses': street_addresses,
    })
    return extracted_addresses_df

In [4]:
def extract_addresses_with_vlookup_one(dataframe, vlookup_column, address_column):
    
    filtered_df = dataframe[dataframe[vlookup_column] == 1]
    result_df = filtered_df[[address_column]].copy()

    # Extract the indices of the selected rows as a list
    index_list = filtered_df.index.tolist()

    # Add a new 'Index' column to the result DataFrame
    result_df['Index'] = index_list

    return result_df

In [5]:
checkInc = extract_addresses_with_vlookup_one(df, vlookup_column='Lookup', address_column='FULL_ADDRESS')
index_df = checkInc.copy()
index_df.drop('FULL_ADDRESS', axis=1, inplace=True)

checkInc = extract_street_addresses(checkInc, 'FULL_ADDRESS')
index_df.reset_index(drop=True, inplace=True)
df_Check = checkInc.join(index_df)
df_Check.drop('Extracted Street Addresses', axis=1, inplace=True)
df_Full = extract_street_addresses(df, 'FULL_ADDRESS')
checkIndex = df_Check['Index'].tolist()

print(checkIndex)
print(df_Full)

[87, 216, 219, 256, 409, 444, 494, 761, 943, 981, 1012, 1091, 1092, 1093, 1094, 1099, 1100, 1101, 1102, 1107, 1108, 1109, 1110, 1115, 1116, 1117, 1118, 1123, 1124, 1125, 1126, 1131, 1132, 1133, 1134, 1139, 1140, 1141, 1142, 1147, 1148, 1149, 1150, 1155, 1156, 1157, 1158, 1163, 1164, 1165, 1166, 1171, 1172, 1173, 1174, 1179, 1180, 1181, 1182, 1187, 1188, 1189, 1190, 1195, 1196, 1197, 1198, 1203, 1204, 1205, 1210, 1211, 1212, 1288, 1295, 1373, 1374, 1578, 1931, 1932, 1933, 1934, 1935, 1936, 2084, 2137, 2214, 2289, 2383, 2389, 2416, 2527, 2605, 2618, 2628, 2704, 2823, 2869, 2887, 2979, 2983, 3074, 3091, 3171, 3179, 3281, 3320, 3325, 3354, 3500, 3580, 3584, 3595, 3622, 3658, 3682, 3698, 3911, 3912, 3942, 3972, 3974, 3982, 4019, 4033, 4037, 4038, 4039, 4069, 4115, 4129, 4154, 4156, 4160, 4250, 4258, 4341, 4348, 4375, 4486, 4490, 4533, 4547, 4573, 4644, 4657, 4683, 4685, 4741, 4838, 4841, 4875, 4880, 4920, 4922, 4927, 4963, 4997]
     Extracted Street Addresses
0                     0 COAL R

In [6]:
df_Full['Incremental'] = np.nan

for i in checkIndex:
    address_Main = df_Full.loc[i,'Extracted Street Addresses']
    address_Above = df_Full.loc[i-1,'Extracted Street Addresses']
    address_Below = df_Full.loc[i+1,'Extracted Street Addresses']
    
    if(address_Main == address_Below or address_Main == address_Above):
        df_Full.loc[i,'Incremental'] = 0
    else:
        df_Full.loc[i,'Incremental'] = 1


In [7]:
print(df_Full)
df_Full.drop('Extracted Street Addresses', axis = 1, inplace = True)
df.drop('Incremental', axis=1, inplace = True)
print(df_Full)
merged_df = df.join(df_Full)
print(merged_df)

     Extracted Street Addresses  Incremental
0                     0 COAL RD          NaN
1                 0 NEW SERVICE          NaN
2                 0 NEW SERVICE          NaN
3                 0 NEW SERVICE          NaN
4               0 PATTERSON TRL          NaN
...                         ...          ...
4994           1039 PRESTON AVE          NaN
4995      1039 S WINCHESTER AVE          NaN
4996               1039 TORI CT          NaN
4997               1039 TORI CT          0.0
4998             1039 TUPELO CT          NaN

[4999 rows x 2 columns]
      Incremental
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
...           ...
4994          NaN
4995          NaN
4996          NaN
4997          0.0
4998          NaN

[4999 rows x 1 columns]
        FULLLOCHID                                     FULL_ADDRESS  \
0     7.970000e+18  0 COAL RD, STUARTS DRAFT, VA 24477 COAL RD CELL   
1     8.210000e+18              0 NEW SERVICE, EAGLE

In [8]:
#Save DataFrame to CSV file

merged_df.to_csv('VA_post_dig_test_output.csv', index=False)

